# --- Day 4: Passport Processing ---

## Imports

In [26]:
import tensorflow as tf

## Part 1

In [126]:
@tf.function
def solve(fname):
    s = tf.io.read_file(fname)
    s = tf.strings.split(s, "\n\n")
    s = tf.map_fn(
        lambda s: tf.strings.regex_replace(s, '\n', ' '),
        s
    )
    s = tf.strings.split(s, ' ')
    s = tf.strings.split(s, ':')
    
    cid_included = tf.math.count_nonzero(s.to_tensor()[:,:,0] == "cid", axis=1)

    # Screw CID    
    passport_lens = tf.map_fn(
        lambda s: s.bounding_shape()[0],
        s,
        fn_output_signature=tf.TensorSpec(shape=(), dtype=tf.int64)
    )

    return tf.math.count_nonzero(passport_lens-cid_included == 7)
        

solve('day4.txt')

<tf.Tensor: shape=(), dtype=int64, numpy=204>

In [127]:
%%timeit
solve('day4.txt')

768 ms ± 21.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [15]:
@tf.function(experimental_compile=True)
def solve2(forest_part):
    return tf.reduce_prod([
        solve(forest_part, r=1, d=1),
        solve(forest_part, r=3, d=1),
        solve(forest_part, r=5, d=1),
        solve(forest_part, r=7, d=1),
        solve(forest_part, r=1, d=2),
    ])

In [16]:
%%timeit
solve2(forest_part)

170 µs ± 45 µs per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [18]:
import datetime

In [19]:
# Set up logging.
stamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
logdir = 'logs/func/%s' % stamp

# Bracket the function call with
# tf.summary.trace_on() and tf.summary.trace_export().
tf.profiler.experimental.start(logdir)
solve2(forest_part)
tf.profiler.experimental.stop()